In [1]:
## import stuff
!pip install tmtoolkit
!pip install --upgrade gensim
#!pip install numpy=1.19.0
#!pip install scipy==1.4.1
#!pip install folium==0.2.1
#!pip install imgaug==0.2.7
import logging
import os
import time
import warnings
import copy
import re
import gensim
import gensim.corpora as corpora
import matplotlib.pyplot as plt
import nltk
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from gensim.matutils import corpus2csc, Sparse2Corpus
from gensim.models import LdaModel, LdaMulticore
from gensim.models.wrappers import LdaMallet
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import datapath
from tmtoolkit.topicmod import evaluate, tm_lda
from sklearn.feature_extraction.text import CountVectorizer
import logging
import os
from gensim import corpora, models, utils
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

     |████████████████████████████████| 7.2MB 5.2MB/s 
     |████████████████████████████████| 14.5MB 244kB/s 
     |████████████████████████████████| 10.4MB 40.7MB/s 
     |████████████████████████████████| 25.9MB 125kB/s 
     |████████████████████████████████| 11.6MB 60.6MB/s 
     |████████████████████████████████| 112kB 58.2MB/s 
     |████████████████████████████████| 1.1MB 53.0MB/s 
  Created wheel for globre: filename=globre-0.1.5-cp36-none-any.whl size=19528 sha256=a2788d67130fe5f607e5dc5c1036494cdf6f7c6f1b9bc1f140d802fd9028ce0f
  Stored in directory: /root/.cache/pip/wheels/c4/3f/7f/be546f6d880521a8a7a1fe84cbe135fa859be63084682d3292
Successfully built globre
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompat

     |████████████████████████████████| 24.2MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
## import the file manually (as it is > 25 MB)
from google.colab import files
uploaded = files.upload()

Saving data_clean.csv to data_clean.csv


In [ ]:
# Dataset is now stored in a Pandas Dataframe
import pandas as pd
import io
df_train = pd.read_csv(io.BytesIO(uploaded['data_clean.csv']))

In [ ]:
# count vectorizer 
from sklearn.feature_extraction.text import CountVectorizer
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=2)

X = bigram_vectorizer.fit_transform(df_train['clean_text'].tolist())

term2id = bigram_vectorizer.vocabulary_

In [ ]:
# more stuff 
from gensim.matutils import corpus2csc, Sparse2Corpus
import gensim.corpora as corpora
corpus = Sparse2Corpus(X, documents_columns=False)
dictionary = corpora.Dictionary.from_corpus(corpus, id2word= {v:k for (k, v) in term2id.items()})


2020-12-08 12:29:07,167 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-12-08 12:29:09,973 : INFO : adding document #10000 to Dictionary(0 unique tokens: [])
2020-12-08 12:29:13,013 : INFO : adding document #20000 to Dictionary(0 unique tokens: [])
2020-12-08 12:29:16,099 : INFO : adding document #30000 to Dictionary(0 unique tokens: [])
2020-12-08 12:29:18,361 : INFO : adding document #40000 to Dictionary(0 unique tokens: [])
2020-12-08 12:29:21,711 : INFO : built Dictionary(851321 unique tokens: ['advantage', 'raspi', 'wifi', 'ble', 'zigbee']...) from 40576 documents (total 17339117 corpus positions)


In [ ]:
# filter dictionary
import copy
filter_dict = copy.deepcopy(dictionary)
filter_dict.filter_extremes(no_below=5, no_above=0.4)

2020-12-08 12:29:33,505 : INFO : discarding 751321 tokens: [('aa aaa', 12), ('aa alkaline', 2), ('aa batterie', 2), ('aa bb', 2), ('aa cell', 3), ('aa d', 2), ('aa last', 4), ('aa lithium', 6), ('aa month', 3), ('aa power', 2)]...
2020-12-08 12:29:33,507 : INFO : keeping 100000 tokens which were in no less than 5 and no more than 16230 (=40.0%) documents
2020-12-08 12:29:34,072 : INFO : resulting dictionary: Dictionary(100000 unique tokens: ['advantage', 'raspi', 'wifi', 'ble', 'zigbee']...)


In [ ]:
# save stuff
#tmp_file = datapath('vocabulary\\test_nb5_na04')
#filter_dict.save(tmp_file)

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', 
                                    vocabulary={term:id_ for (id_, term) in filter_dict.items()})

train_bigram = bigram_vectorizer.fit(df_train['clean_text'].tolist())

In [ ]:
X = bigram_vectorizer.transform(df_train['clean_text'].tolist())

corpus = Sparse2Corpus(X, documents_columns=False)

In [ ]:
## LdaGensim basic function: 
import timeit
def LdaGensim_topics(dictionary, corpus, limit, start, step, alpha, beta):
    '''
    Parameters:
    ---------
    dictionary: Gensim dictionary
    corpus: Gensim corpus
    limit: max num of topics (end at limit -1)
    start: min number of topics
    step: increment between each integer
    alpha: list of prior on the per-document topic distribution.
    eta: list of prior on the per-topic word distribution. 

    Returns:
    ---------
    model_list: list of LDA topic
    '''

    # initialize an empty dictionary
    models = {}

    for a in alpha: 
        print('Running model with alpha={}/k'.format(a))
        for b in beta:
            print('Running model with beta={}'.format(b))
            for num_topics in range(start, limit, step):
                print('Running model with number of topics (k): ', num_topics)
                start = timeit.default_timer()
                model = LdaModel(corpus=corpus, 
                                 id2word=dictionary,
                                 num_topics=num_topics, random_state=123, 
                                 passes=10, 
                                 alpha=[a]*num_topics, 
                                 eta=b,
                                 eval_every = 50,
                                 per_word_topics=True)
                stop = timeit.default_timer()
                print(f'it took {(stop-start)/60} min')
                name = "a{0}_b{1}_k{2}".format(a, b, num_topics)
                models[name] = model

                print('\n')

    return models

In [ ]:
#Run it (only up to 100): 
models = LdaGensim_topics(dictionary=filter_dict, corpus=corpus, 
                                                         start=20, limit=21, step=5, 
                                                         alpha = [1], 
                                                         beta = [1])

2020-12-08 12:36:21,697 : INFO : using serial LDA version on this node


Running model with alpha=1/k
Running model with beta=1
Running model with number of topics (k):  20


2020-12-08 12:36:21,961 : INFO : running online (multi-pass) LDA training, 20 topics, 10 passes over the supplied corpus of 40576 documents, updating model once every 2000 documents, evaluating perplexity every 40576 documents, iterating 50x with a convergence threshold of 0.001000
2020-12-08 12:36:22,174 : INFO : PROGRESS: pass 0, at document #2000/40576
2020-12-08 12:36:25,406 : INFO : merging changes from 2000 documents into a model of 40576 documents
2020-12-08 12:36:25,528 : INFO : topic #18 (1.000): 0.005*"hub" + 0.004*"hue" + 0.004*"system" + 0.004*"turn" + 0.003*"bulb" + 0.003*"run" + 0.003*"wave" + 0.003*"set" + 0.003*"sensor" + 0.003*"house"
2020-12-08 12:36:25,531 : INFO : topic #17 (1.000): 0.004*"wave" + 0.004*"system" + 0.004*"door" + 0.003*"run" + 0.003*"hub" + 0.003*"lock" + 0.003*"wire" + 0.002*"buy" + 0.002*"sensor" + 0.002*"may"
2020-12-08 12:36:25,534 : INFO : topic #3 (1.000): 0.005*"hub" + 0.004*"system" + 0.004*"turn" + 0.004*"wave" + 0.003*"sensor" + 0.003*"door

it took 8.9954497375 min




In [ ]:
#Run it (only up to 100): 
models = LdaGensim_Multi(dictionary=filter_dict, corpus=corpus, 
                                                         start=5, limit=20, step=5, ## limit = 101
                                                         alpha = [0.01, 0.1, 1, 10], 
                                                         beta = [0.01, 0.1, 1, 10])

Running model with alpha=0.01/k
Running model with beta=0.01
Running model with number of topics (k):  5


/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
Process ForkPoolWorker-25:
Traceback (most recent call last):
Process ForkPoolWorker-24:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 9

KeyboardInterrupt: ignored

  File "/usr/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
KeyboardInterrupt
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **se

In [ ]:
def corpus2token_text(corpus, dictionary):
  nested_doc = []
  texts = []
  for doc in corpus:
      nested_doc.append([[dictionary[k]]*v for (k, v) in doc])
  for doc in nested_doc:
      texts.append([item for sublist in doc for item in sublist])
  return texts

In [ ]:
txt = corpus2token_text(corpus, filter_dict)

In [ ]:
## save the models: 
models

{'a1_b1_k20': <gensim.models.ldamodel.LdaModel at 0x7fca9929b6a0>}

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Coherence Score
cohr_lda_multicore_model2 = CoherenceModel(model = models['a1_b1_k20'], texts = txt, dictionary = filter_dict, coherence='c_v').get_coherence()

print('\nCoherence Score: ', cohr_lda_multicore_model2)

Streaming af output blev afkortet til de sidste 5000 linjer.
2020-12-08 12:48:33,512 : INFO : WordOccurrenceAccumulator accumulated stats from 3975000 documents
2020-12-08 12:48:33,548 : INFO : WordOccurrenceAccumulator accumulated stats from 3976000 documents
2020-12-08 12:48:33,580 : INFO : WordOccurrenceAccumulator accumulated stats from 3977000 documents
2020-12-08 12:48:33,604 : INFO : WordOccurrenceAccumulator accumulated stats from 3978000 documents
2020-12-08 12:48:33,624 : INFO : WordOccurrenceAccumulator accumulated stats from 3979000 documents
2020-12-08 12:48:33,656 : INFO : WordOccurrenceAccumulator accumulated stats from 3980000 documents
2020-12-08 12:48:33,689 : INFO : WordOccurrenceAccumulator accumulated stats from 3981000 documents
2020-12-08 12:48:33,726 : INFO : WordOccurrenceAccumulator accumulated stats from 3982000 documents
2020-12-08 12:48:33,751 : INFO : WordOccurrenceAccumulator accumulated stats from 3983000 documents
2020-12-08 12:48:33,777 : INFO : WordOc


Coherence Score:  0.25204050487066904
